## Convert VIA annotations export .csv file to RetinaNet .csv format

#### Greg has adapted this from the script Patrick and Maddie used

In [39]:
# import necessary modules
import os
import csv
import random

# set pseudo-random values for replicability
random.seed(1)

# set directory to root where files live
os.chdir('C:/Users/Greg/Documents/GitHub/GreySealCNN')

# set path to VIA exported CSCV file
via_path = 'data/via_SealCNN_TrainingData.csv'

#### Pull names of data files, shuffle, and split into 3 datasets: training, testing, validation
###### Script adapted from Maddie's version

In [40]:
# pull images from directory where they live into a list
path_images = 'data/Hay Island tiles'
image_list = [f for f in os.listdir(path_images) if f.endswith('.png')]
#print(image_list[:5])

# shuffle the list randomly and get total count
random.shuffle(image_list)
total_count = len(image_list)

# set indices for breaking up the total dataset into TTV parts
test_fraction = 0.1
valid_fraction = 0.04
train_fraction = 0.86
if (sum([test_fraction, valid_fraction, train_fraction]) != 1.0):
   raise NameError("fractions should add up to 1")

test_index = int(total_count * test_fraction)
valid_index = int(total_count * (test_fraction + valid_fraction))

# use indices to break up dataset into TTV parts
test_dataset = image_list[:test_index]
valid_dataset = image_list[test_index:valid_index]
train_dataset = image_list[valid_index:]
print(len(test_dataset), len(valid_dataset), len(train_dataset))

22 9 192


#### The following loop pulls each annotation, line-by-line, from the VIA exported CSV, extracts the necessary information, reformats it into the format that RetinaNet requires (https://github.com/fizyr/keras-retinanet#annotations-format), end then reassembles a new CSV line-by-line that RetinaNet can receive

In [41]:
#GDL has overhauled this code to ingest annotations exported as CSV from VIA 2.0.10

# Create blank variable for each annotations list as we build it
image_annotations_train = []
image_annotations_test = []
image_annotations_valid = []

with open(via_path, "r") as f:
    reader = csv.reader(f, delimiter=",")
    for line in reader: 
        # output we want
        # path/to/image.jpg,x1,y1,x2,y2,class_name
        # /data/imgs/img_001.jpg,837,346,981,456,cow
        if 'filename' in line[0]:
            # bypassing comments in csv
            continue
        if '{}' in line[5]:
            #bypassing empty images
            continue
        filename = line[0]
        
        # pulling from column named "region_shape_attributes"
        box_entry = list(str(line[5]).strip('}{').split(','))
        box_entry = [i.split(':')[1] for i in box_entry]
        # strip brackets, split and get only the values we care about, then convert all the string to int 
        top_left_x, top_left_y, width, height = list(map(int,list(map(float, box_entry[1:5]))))
        if width == 0 or height == 0:
            continue
            # skip tiny/empty boxes
        
        # convert from "top left and width/height" to "x and y values at each corner of the box"
        if top_left_x < 0:
            top_left_x = 1
        if top_left_y < 0:
            top_left_y = 1
        x1 = top_left_x
        x2 = top_left_x + width
        y1 = top_left_y
        y2 = top_left_y + height 
        
        # pulling from column named "region_attributes" to get class names
        name = list(str(line[6]).strip('}{').split(':'))[1].strip('"')

        # skip unknown class, in this case. Might be useful in other applications though, e.g. total count
        if name == "Unknown":
            continue
            

        # create the csv row
        new_row = []
        new_row.append(filename)
        new_row.append(x1)
        new_row.append(y1)
        new_row.append(x2)
        new_row.append(y2)
        new_row.append(name)
        
        # append the row to the correct CSV, depending on which part it belongs to
        if filename in train_dataset:
            image_annotations_train.append(new_row)
        elif filename in test_dataset:
            image_annotations_test.append(new_row)
        else:
            image_annotations_valid.append(new_row)

### Output annotations.csv and classes.csv

In [42]:
with open('data/annotations_train.csv', 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_train)

with open('data/annotations_test.csv', 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_test)

with open('data/annotations_valid.csv', 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_valid)

In [43]:
# if we were trying to generalize this, we'd be tracking new class names as they show up,
# assigning values and automating construction of our list of classes...
# but with only three classes, screw it, we'll just write them manually

# also, note that "unknown" ambiguous cases have been excluded
detection_classes = [["Adult", 0], ["Pup", 1]]
with open('data/classes.csv', 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerows(detection_classes)

#### Not a code issue, but I recommend manually checking each annotations document to make sure that there are a reasonable number of annotations in each dataset; it is possible that the random breakdown could pick a cluster of "empty" images for validation or training. Should not be an issue in this specific case because I've checked it for our random seed, but in future applications this is a good idea.